In [3]:
%cd '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src'

/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src


In [ ]:
!pip install transformers
!pip3 install git+https://github.com/ernie-mlg/rpunct.git
!pip install git+https://github.com/tagucci/pythonrouge.git
!pip install rouge-metric
from rouge_metric import PyRouge
# from tqdm.auto import tqdm
from tqdm import trange, tqdm
import os
import sys
import requests
import json
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ernie-mlg/rpunct.git to /tmp/pip-req-build-1glu061y
  Running command git clone --filter=blob:none --quiet https://github.com/ernie-mlg/rpunct.git /tmp/pip-req-build-1glu061y
  Resolved https://github.com/ernie-mlg/rpunct.git to commit 4c378cf570f05aa58c7d3dbb896b51ca2d95fde6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━

### Load data

In [ ]:
data_dir = '../data/'
data_list = os.listdir(data_dir)
print(data_list)

['cnn10.json', 'cnn10_full.json', 'ramsay10.json', 'ted.json', 'reviews.json', 'podcasts.json', 'summary']


### Data: CNN Full

In [ ]:
data_file = 'cnn10_full.json'
data_path = data_dir + data_file
data_name = data_file[:-5]
save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
print("Data File:", data_name)
df = pd.read_json(data_path)
df.head()

Data File: cnn10_full


,id,text_captions,fused_captions,summary
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...
1,Kc_GMYHJNFQ,[music] Thank you hello everyone! Wonderful! W...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,..."
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...


#### Summarisation

In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Pegasus
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

# MT5 
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

print(df.head())
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]
print(df.describe())

df.to_excel(save_data_path, index=False)

  0%|          | 0/86 [00:00<?, ?it/s]

100%|██████████| 86/86 [04:58<00:00,  3.47s/it]


/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


100%|██████████| 86/86 [05:38<00:00,  3.94s/it]


            id                                      text_captions  \
0  8GgaCgjAFLM  [music] Thank you Hello lovely people Happy! F...   
1  Kc_GMYHJNFQ  [music] Thank you hello everyone! Wonderful! W...   
2  wPdiuj2JwEU  [music] Thank you Hey there everyone Terrific!...   
3  H07TJT3VpSY  [music] Thank you! What's up Sunshine, Welcome...   
4  U4WxR7hXfCQ  [music] Thank you! What's up lovely people? Th...   

                                      fused_captions  \
0  [{'text': '[Music]', 'start': 0.0, 'duration':...   
1  [{'text': '[Music]', 'start': 0.0, 'duration':...   
2  [{'text': '[Music]', 'start': 0.0, 'duration':...   
3  [{'text': '[Music]', 'start': 0.0, 'duration':...   
4  [{'text': '[Music]', 'start': 0.0, 'duration':...   

                                             summary  \
0   It's been nearly three years since the start ...   
1   Some farmers are not harvesting any tomatoes,...   
2   CNN Business Correspondent Rahel Solomon has ...   
3   Friday marked one ye

#### Evaluation

In [ ]:
eval_df = pd.read_excel(save_data_path)
eval_df.head()

rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

print(eval_df.head())
eval_df.to_excel(save_data_path, index=False)

pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

print(pegasus_rouge_df.head())
print(pegasus_rouge_df.describe())

mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

100%|██████████| 86/86 [00:07<00:00, 11.21it/s]


            id                                      text_captions  \
0  8GgaCgjAFLM  [music] Thank you Hello lovely people Happy! F...   
1  Kc_GMYHJNFQ  [music] Thank you hello everyone! Wonderful! W...   
2  wPdiuj2JwEU  [music] Thank you Hey there everyone Terrific!...   
3  H07TJT3VpSY  [music] Thank you! What's up Sunshine, Welcome...   
4  U4WxR7hXfCQ  [music] Thank you! What's up lovely people? Th...   

                                      fused_captions  \
0  [{'text': '[Music]', 'start': 0.0, 'duration':...   
1  [{'text': '[Music]', 'start': 0.0, 'duration':...   
2  [{'text': '[Music]', 'start': 0.0, 'duration':...   
3  [{'text': '[Music]', 'start': 0.0, 'duration':...   
4  [{'text': '[Music]', 'start': 0.0, 'duration':...   

                                             summary  \
0   It's been nearly three years since the start ...   
1   Some farmers are not harvesting any tomatoes,...   
2   CNN Business Correspondent Rahel Solomon has ...   
3   Friday marked one ye

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,0.235222,0.053943,0.013971,0.134614,0.047813,0.042261,0.075317
std,0.047015,0.039651,0.030168,0.036272,0.015656,0.031883,0.032706
min,0.116402,0.000000,0.000000,0.063158,0.021792,0.006522,0.025408
25%,0.202671,0.024279,0.000000,0.111749,0.037983,0.022813,0.054046
50%,0.233721,0.044569,0.000000,0.129444,0.044252,0.036146,0.070953
75%,0.262757,0.073977,0.011544,0.155345,0.054048,0.046033,0.083919
max,0.356757,0.196319,0.176101,0.266667,0.118488,0.183648,0.208768


### Data: Podcasts

In [ ]:
data_file = 'podcasts.json'
data_path = data_dir + data_file
data_name = data_file[:-5]
save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
print("Data File:", data_name)
df = pd.read_json(data_path)
df.head()

Data File: podcasts


,id,text_captions,fused_captions,summary
0,9V7lvEkslV4,*glass smashes.* *More Glass smashes* A: Hey E...,"[{'text': '*Glass smashes.*', 'start': 8.52, '...",Game Grumps 10 Minute Power Hour is unscripte...
1,VWCT8N267ss,Dan: It's time to move on to segment number tw...,[{'text': 'Dan: It's time to move on to segmen...,Dan: It's time to move on to segment number t...
2,17fw6p7Z1WU,*squelch squelch squelch* Arin: Are You rollin...,"[{'text': '*squelch squelch squelch*', 'start'...",Dan's Robot: Mm SoMeThIn'In'S CoOkIn' MuSt Be...
3,EyNnFiGmnkA,Dan: Have We begun? Arin: Welcome to the Zehn ...,[{'text': 'Dan: Have we begun? Arin: Welcome t...,"Dan: ""We're gonna be so fucking cute, dude. O..."
4,XZvPNPFKPRY,*opening music* Dan: What am I looking at? Ari...,"[{'text': '*opening music*', 'start': 4.18, 'd...",The 10 Minute Power Hour is hosted by Dan and...


#### Summarisation

In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Pegasus
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

# MT5 
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

print(df.head())
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]
print(df.describe())

df.to_excel(save_data_path, index=False)

100%|██████████| 93/93 [05:33<00:00,  3.58s/it]
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 93/93 [06:40<00:00,  4.30s/it]


            id                                      text_captions  \
0  9V7lvEkslV4  *glass smashes.* *More Glass smashes* A: Hey E...   
1  VWCT8N267ss  Dan: It's time to move on to segment number tw...   
2  17fw6p7Z1WU  *squelch squelch squelch* Arin: Are You rollin...   
3  EyNnFiGmnkA  Dan: Have We begun? Arin: Welcome to the Zehn ...   
4  XZvPNPFKPRY  *opening music* Dan: What am I looking at? Ari...   

                                      fused_captions  \
0  [{'text': '*Glass smashes.*', 'start': 8.52, '...   
1  [{'text': 'Dan: It's time to move on to segmen...   
2  [{'text': '*squelch squelch squelch*', 'start'...   
3  [{'text': 'Dan: Have we begun?
Arin: Welcome t...   
4  [{'text': '*opening music*', 'start': 4.18, 'd...   

                                             summary  \
0   Game Grumps 10 Minute Power Hour is unscripte...   
1   Dan: It's time to move on to segment number t...   
2   Dan's Robot: Mm SoMeThIn'In'S CoOkIn' MuSt Be...   
3   Dan: "We're gonna be

#### Evaluation

In [ ]:
eval_df = pd.read_excel(save_data_path)
eval_df.head()

rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

print(eval_df.head())
eval_df.to_excel(save_data_path, index=False)

pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

print(pegasus_rouge_df.head())
print(pegasus_rouge_df.describe())

mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

100%|██████████| 93/93 [00:04<00:00, 20.53it/s]


            id                                      text_captions  \
0  9V7lvEkslV4  *glass smashes.* *More Glass smashes* A: Hey E...   
1  VWCT8N267ss  Dan: It's time to move on to segment number tw...   
2  17fw6p7Z1WU  *squelch squelch squelch* Arin: Are You rollin...   
3  EyNnFiGmnkA  Dan: Have We begun? Arin: Welcome to the Zehn ...   
4  XZvPNPFKPRY  *opening music* Dan: What am I looking at? Ari...   

                                      fused_captions  \
0  [{'text': '*Glass smashes.*', 'start': 8.52, '...   
1  [{'text': "Dan: It's time to move on to segmen...   
2  [{'text': '*squelch squelch squelch*', 'start'...   
3  [{'text': 'Dan: Have we begun?\nArin: Welcome ...   
4  [{'text': '*opening music*', 'start': 4.18, 'd...   

                                             summary  \
0   Game Grumps 10 Minute Power Hour is unscripte...   
1   Dan: It's time to move on to segment number t...   
2   Dan's Robot: Mm SoMeThIn'In'S CoOkIn' MuSt Be...   
3   Dan: "We're gonna be

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,0.178622,0.033181,0.003298,0.098866,0.030478,0.023139,0.049593
std,0.039998,0.022733,0.006784,0.023042,0.009418,0.012702,0.015645
min,0.095522,0.000000,0.000000,0.053731,0.012886,0.001990,0.018182
25%,0.147186,0.016000,0.000000,0.079787,0.023788,0.013793,0.038781
50%,0.180758,0.029326,0.000000,0.098214,0.029673,0.020513,0.047423
75%,0.203947,0.047619,0.000000,0.117647,0.035810,0.030423,0.059152
max,0.280000,0.129032,0.032787,0.154762,0.056887,0.075410,0.108992


### Data: Ramsay

In [ ]:
data_file = 'ramsay10.json'
data_path = data_dir + data_file
data_name = data_file[:-5]
save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
print("Data File:", data_name)
df = pd.read_json(data_path)
df.head()

Data File: ramsay10


,id,text_captions,fused_captions,summary
0,7cVg0hYi5io,[music] right. Guess what? Everyone in my hous...,"[{'text': '[Music]', 'start': 1.83999999999999...",This is an amazing cauliflower steak that wil...
1,99ND_B5kMd4,"Happy New Year Now! Come on, let's be honest, ...",[{'text': 'happy new year now come on let's be...,This is a beautiful flank steak and it's a bi...
2,LB9KDxAOMvI,"[music] Now come on, we've all got a sweet too...","[{'text': '[Music]', 'start': 1.83999999999999...",Rhubarb tarts are so easy for a Sunday brunch...
3,a-2n_g4AdDM,"[music] Now come on, let's be honest, who does...","[{'text': '[Music]', 'start': 1.83999999999999...",Ramsay shows you how to make a delicious spic...
4,tpikKCfm-kU,"[music] Now, throughout my travels across Asia...","[{'text': '[Music]', 'start': 1.83999999999999...",Thai style lettuce wrap is a hodgepodge smorg...


#### Summarisation

In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Pegasus
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

# MT5 
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

print(df.head())
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]
print(df.describe())

df.to_excel(save_data_path, index=False)

100%|██████████| 31/31 [01:52<00:00,  3.62s/it]
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 31/31 [02:31<00:00,  4.88s/it]


            id                                      text_captions  \
0  7cVg0hYi5io  [music] right. Guess what? Everyone in my hous...   
1  99ND_B5kMd4  Happy New Year Now! Come on, let's be honest, ...   
2  LB9KDxAOMvI  [music] Now come on, we've all got a sweet too...   
3  a-2n_g4AdDM  [music] Now come on, let's be honest, who does...   
4  tpikKCfm-kU  [music] Now, throughout my travels across Asia...   

                                      fused_captions  \
0  [{'text': '[Music]', 'start': 1.83999999999999...   
1  [{'text': 'happy new year now come on let's be...   
2  [{'text': '[Music]', 'start': 1.83999999999999...   
3  [{'text': '[Music]', 'start': 1.83999999999999...   
4  [{'text': '[Music]', 'start': 1.83999999999999...   

                                             summary  \
0   This is an amazing cauliflower steak that wil...   
1   This is a beautiful flank steak and it's a bi...   
2   Rhubarb tarts are so easy for a Sunday brunch...   
3   Ramsay shows you how

#### Evaluation

In [ ]:
eval_df = pd.read_excel(save_data_path)
eval_df.head()

rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

print(eval_df.head())
eval_df.to_excel(save_data_path, index=False)

pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

print(pegasus_rouge_df.head())
print(pegasus_rouge_df.describe())

mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

100%|██████████| 31/31 [00:00<00:00, 40.34it/s]


            id                                      text_captions  \
0  7cVg0hYi5io  [music] right. Guess what? Everyone in my hous...   
1  99ND_B5kMd4  Happy New Year Now! Come on, let's be honest, ...   
2  LB9KDxAOMvI  [music] Now come on, we've all got a sweet too...   
3  a-2n_g4AdDM  [music] Now come on, let's be honest, who does...   
4  tpikKCfm-kU  [music] Now, throughout my travels across Asia...   

                                      fused_captions  \
0  [{'text': '[Music]', 'start': 1.83999999999999...   
1  [{'text': "happy new year now come on let's be...   
2  [{'text': '[Music]', 'start': 1.83999999999999...   
3  [{'text': '[Music]', 'start': 1.83999999999999...   
4  [{'text': '[Music]', 'start': 1.83999999999999...   

                                             summary  \
0   This is an amazing cauliflower steak that wil...   
1   This is a beautiful flank steak and it's a bi...   
2   Rhubarb tarts are so easy for a Sunday brunch...   
3   Ramsay shows you how

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.224234,0.049488,0.011478,0.132242,0.049555,0.038079,0.070127
std,0.053491,0.043188,0.031065,0.034733,0.017680,0.031326,0.033150
min,0.137255,0.000000,0.000000,0.082902,0.027987,0.003478,0.032258
25%,0.182512,0.016061,0.000000,0.114455,0.036069,0.019216,0.047554
50%,0.217195,0.038168,0.000000,0.121212,0.048701,0.029457,0.062081
75%,0.268487,0.073149,0.000000,0.145796,0.060007,0.047474,0.086754
max,0.327684,0.171429,0.152047,0.271186,0.112163,0.163743,0.192233


### Data: Reviews

In [ ]:
data_file = 'reviews.json'
data_path = data_dir + data_file
data_name = data_file[:-5]
save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
print("Data File:", data_name)
df = pd.read_json(data_path)
df.head()

Data File: reviews


,id,text_captions,fused_captions,summary
0,zhoTX0RRXPQ,"(upbeat techno music) - Matte Black glass, alu...","[{'text': '(upbeat techno music)', 'start': 0....",The Samsung Galaxy S23 Ultra has a Snapdragon...
1,3yEP4ooj1WI,I was kind of wondering how long it would be b...,[{'text': 'I was kind of wondering how long it...,"This phone has a Snapdragon 8, Gen 2 with 8 g..."
2,gvqZCMGjh3s,Foreign [Music] [Applause] [Music] Look who we...,"[{'text': 'foreign', 'start': 0.48, 'duration'...",The original homepod came out in 2018. It was...
3,cneoANZKBGk,"(logo whirring) - Hey, what's up? MKBHD here. ...","[{'text': '(logo whirring)', 'start': 0.017, '...",The base M2 Mac mini is a game changer becaus...
4,tmGDx9hVWwo,(upbeat music) (upbeat Music continues) - So T...,"[{'text': '(upbeat music)', 'start': 0.38, 'du...",Apple updated the 14 inch and 16 inch MacBook...


#### Summarisation

In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Pegasus
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

# MT5 
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

print(df.head())
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]
print(df.describe())

df.to_excel(save_data_path, index=False)

100%|██████████| 108/108 [06:33<00:00,  3.64s/it]
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 108/108 [08:04<00:00,  4.48s/it]


            id                                      text_captions  \
0  zhoTX0RRXPQ  (upbeat techno music) - Matte Black glass, alu...   
1  3yEP4ooj1WI  I was kind of wondering how long it would be b...   
2  gvqZCMGjh3s  Foreign [Music] [Applause] [Music] Look who we...   
3  cneoANZKBGk  (logo whirring) - Hey, what's up? MKBHD here. ...   
4  tmGDx9hVWwo  (upbeat music) (upbeat Music continues) - So T...   

                                      fused_captions  \
0  [{'text': '(upbeat techno music)', 'start': 0....   
1  [{'text': 'I was kind of wondering how long it...   
2  [{'text': 'foreign', 'start': 0.48, 'duration'...   
3  [{'text': '(logo whirring)', 'start': 0.017, '...   
4  [{'text': '(upbeat music)', 'start': 0.38, 'du...   

                                             summary  \
0   The Samsung Galaxy S23 Ultra has a Snapdragon...   
1   This phone has a Snapdragon 8, Gen 2 with 8 g...   
2   The original homepod came out in 2018. It was...   
3   The base M2 Mac mini

#### Evaluation

In [ ]:
eval_df = pd.read_excel(save_data_path)
eval_df.head()

rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

print(eval_df.head())
eval_df.to_excel(save_data_path, index=False)

pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

print(pegasus_rouge_df.head())
print(pegasus_rouge_df.describe())

mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

100%|██████████| 108/108 [00:07<00:00, 13.72it/s]


            id                                      text_captions  \
0  zhoTX0RRXPQ  (upbeat techno music) - Matte Black glass, alu...   
1  3yEP4ooj1WI  I was kind of wondering how long it would be b...   
2  gvqZCMGjh3s  Foreign [Music] [Applause] [Music] Look who we...   
3  cneoANZKBGk  (logo whirring) - Hey, what's up? MKBHD here. ...   
4  tmGDx9hVWwo  (upbeat music) (upbeat Music continues) - So T...   

                                      fused_captions  \
0  [{'text': '(upbeat techno music)', 'start': 0....   
1  [{'text': 'I was kind of wondering how long it...   
2  [{'text': 'foreign', 'start': 0.48, 'duration'...   
3  [{'text': '(logo whirring)', 'start': 0.017, '...   
4  [{'text': '(upbeat music)', 'start': 0.38, 'du...   

                                             summary  \
0   The Samsung Galaxy S23 Ultra has a Snapdragon...   
1   This phone has a Snapdragon 8, Gen 2 with 8 g...   
2   The original homepod came out in 2018. It was...   
3   The base M2 Mac mini

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000
mean,0.194568,0.038658,0.005194,0.113035,0.033098,0.031479,0.059206
std,0.043232,0.028705,0.014991,0.030181,0.012339,0.022849,0.025152
min,0.116208,0.000000,0.000000,0.057692,0.015409,0.001980,0.029207
25%,0.164343,0.020070,0.000000,0.095238,0.026167,0.018333,0.044330
50%,0.189769,0.033397,0.000000,0.109246,0.031053,0.027305,0.055696
75%,0.216998,0.050158,0.001174,0.121956,0.037092,0.039508,0.069332
max,0.415094,0.200000,0.116505,0.320755,0.116867,0.186408,0.225806


### Data: Ted

In [ ]:
data_file = 'ted.json'
data_path = data_dir + data_file
data_name = data_file[:-5]
save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
print("Data File:", data_name)
df = pd.read_json(data_path)
df.head()

Data File: ted


,id,text_captions,fused_captions,summary
0,MB5IX-np5fE,"For A really long time, I had two mysteries th...","[{'text': 'For a really long time,', 'start': ...",The author has written a book about the cause...
1,5MuIMqhT8DM,"Thank you very much. Well, I Would like to sta...","[{'text': 'Thank you very much.', 'start': 0.8...",A lack of sleep will age a man by a decade in...
2,eIho2S0ZahI,The Human voice: It's the instrument we all pl...,"[{'text': 'The human voice:', 'start': 13.96, ...",The Human voice is the most powerful sound in...
3,Hu4Yvq-g7_Y,Translator: Viviane P. Reviewer: Amanda Chu A ...,[{'text': 'Translator: Viviane P. Reviewer: Am...,"A few years ago, I began to observe something..."
4,ON4iy8hq2hM,"So When you think about a child,, a close frie...","[{'text': 'So when you think about a child, a ...",One in three women and one in four men will e...


#### Summarisation

In [ ]:
def summarize(tokenizer, model, text, device='cuda:0'):

    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt").to(device)
    model.to(device)
    summary_ids = model.generate(inputs["input_ids"], max_new_tokens=120, min_length=90)
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

checkpoint = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Pegasus
df['pegasus_summary'] = ""
for i in trange(len(df)):
    df['pegasus_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

df.head()

# MT5 
checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

df['mT5_summary'] = ""
for i in trange(len(df)):
    df['mT5_summary'].iloc[i] = summarize(tokenizer, model, df['text_captions'].iloc[i])[0]

print(df.head())
df['pegasus_summary'].iloc[0], df['mT5_summary'].iloc[0]
print(df.describe())

df.to_excel(save_data_path, index=False)

100%|██████████| 117/117 [06:59<00:00,  3.59s/it]
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 117/117 [08:28<00:00,  4.34s/it]


            id                                      text_captions  \
0  MB5IX-np5fE  For A really long time, I had two mysteries th...   
1  5MuIMqhT8DM  Thank you very much. Well, I Would like to sta...   
2  eIho2S0ZahI  The Human voice: It's the instrument we all pl...   
3  Hu4Yvq-g7_Y  Translator: Viviane P. Reviewer: Amanda Chu A ...   
4  ON4iy8hq2hM  So When you think about a child,, a close frie...   

                                      fused_captions  \
0  [{'text': 'For a really long time,', 'start': ...   
1  [{'text': 'Thank you very much.', 'start': 0.8...   
2  [{'text': 'The human voice:', 'start': 13.96, ...   
3  [{'text': 'Translator: Viviane P.
Reviewer: Am...   
4  [{'text': 'So when you think about a child,
a ...   

                                             summary  \
0   The author has written a book about the cause...   
1   A lack of sleep will age a man by a decade in...   
2   The Human voice is the most powerful sound in...   
3   A few years ago, I b

#### Evaluation

In [ ]:
eval_df = pd.read_excel(save_data_path)
eval_df.head()

rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

def compute_summary_rouge(pred, gt):
    scores = rouge.evaluate([pred,], [[gt,],])
    return scores

eval_df['pegasus_rouge']=None
eval_df['mT5_rouge'] = None
pegasus_rouge_dict = []
mT5_rouge_dict = []

for i in trange(len(df)):
    temp = compute_summary_rouge(eval_df['pegasus_summary'].iloc[i], eval_df['summary'].iloc[i])
    pegasus_rouge_dict.append(temp)
    eval_df['pegasus_rouge'].iloc[i] = str(temp)

    temp = compute_summary_rouge(eval_df['mT5_summary'].iloc[i], eval_df['summary'].iloc[i])
    mT5_rouge_dict.append(temp)
    eval_df['mT5_rouge'].iloc[i] = str(temp)

print(eval_df.head())
eval_df.to_excel(save_data_path, index=False)

pegasus_rouge_df = pd.DataFrame.from_dict(pegasus_rouge_dict)

for column in pegasus_rouge_df:
    pegasus_rouge_df[column] = pegasus_rouge_df[column].apply(lambda x: x['f'])

print(pegasus_rouge_df.head())
print(pegasus_rouge_df.describe())

mT5_rouge_df = pd.DataFrame.from_dict(mT5_rouge_dict)

for column in mT5_rouge_df:
    mT5_rouge_df[column] = mT5_rouge_df[column].apply(lambda x: x['f'])

mT5_rouge_df.head()
mT5_rouge_df.describe()

100%|██████████| 117/117 [00:05<00:00, 20.63it/s]


            id                                      text_captions  \
0  MB5IX-np5fE  For A really long time, I had two mysteries th...   
1  5MuIMqhT8DM  Thank you very much. Well, I Would like to sta...   
2  eIho2S0ZahI  The Human voice: It's the instrument we all pl...   
3  Hu4Yvq-g7_Y  Translator: Viviane P. Reviewer: Amanda Chu A ...   
4  ON4iy8hq2hM  So When you think about a child,, a close frie...   

                                      fused_captions  \
0  [{'text': 'For a really long time,', 'start': ...   
1  [{'text': 'Thank you very much.', 'start': 0.8...   
2  [{'text': 'The human voice:', 'start': 13.96, ...   
3  [{'text': 'Translator: Viviane P.\nReviewer: A...   
4  [{'text': 'So when you think about a child,\na...   

                                             summary  \
0   The author has written a book about the cause...   
1   A lack of sleep will age a man by a decade in...   
2   The Human voice is the most powerful sound in...   
3   A few years ago, I b

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-w-1.2,rouge-s4,rouge-su4
count,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000
mean,0.183026,0.037452,0.012174,0.108336,0.031869,0.032954,0.058466
std,0.045021,0.037614,0.025774,0.032981,0.013603,0.028751,0.029624
min,0.092105,0.000000,0.000000,0.057018,0.012245,0.003524,0.022929
25%,0.152439,0.013115,0.000000,0.084507,0.022756,0.015652,0.039735
50%,0.177049,0.024691,0.000000,0.097561,0.026690,0.023410,0.050874
75%,0.214925,0.045326,0.010929,0.127517,0.036688,0.040467,0.068670
max,0.374502,0.240964,0.155102,0.262948,0.092537,0.200816,0.229308
